## Volume Change-PyGem

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import date
import collections
import datetime
import os
import xarray as xr

In [ ]:
#All of the climate models used
modelnames = ['BCC-CSM2-MR','CESM2','CESM2-WACCM','EC-Earth3','EC-Earth3-Veg','FGOALS-f3-L','GFDL-ESM4',
              'INM-CM4-8','INM-CM5-0','MPI-ESM1-2-HR','MRI-ESM2-0', 'NorESM2-MM']

SSPs = ['ssp126','ssp245','ssp370','ssp585'] #List of all SSPs in PyGEM
which_ssp = SSPs[0]

alpine_basins = {'RHINE': '6242',
                 'RHONE': '6243',
                 'PO': '6241',
                 'DANUBE':'6202'} ## GRDC Major River Basin identifiers for the 4 alpine basins we can study

test_basin = alpine_basins['RHONE'] 

#Generic filepath to navigate to Drive folder 
fpath0 = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/Lizz Research Stuff'

In [ ]:
import json
def select_glaciers_json(basin='all'):
    '''
    Select glaciers within a basin by MRBID from a json-file,
    which is stored in the data directory.

    Args:
    -----
    basin: str
        String of MRBID or 'all'.

    Returns:
    --------
    If basin is 'all' a list of all relevant glaciers is returned, for
    initiating glacier simulations. If basin is a MRBID the list of glaciers
    within that basin is returned.
    
    Copy of a function written by Erik Holmgren (2022) in holmgren_gha.utils
    '''

    # fpath = './data/rgi_ids_per_basin.json'
    fpath = '/Users/finnwimberly/Library/CloudStorage/GoogleDrive-fwimberly@middlebury.edu/My Drive/Lizz Research Stuff/rgi_ids_per_basin.json'  
    with open(fpath) as f:
        basin_dict = json.load(f)

    if basin.lower() != 'all':
        glacier_list = basin_dict[basin]
    else:
        glacier_list = list(itertools.chain.from_iterable(basin_dict.values()))

    return glacier_list

In [ ]:
#Creating list of glaciers located within a basin using above function
basin_gls = {}
for basin, code in alpine_basins.items():
    basin_gls[basin] = select_glaciers_json(code)

In [ ]:
#Based upon code developed by Lizz Ultee
batches = ['1-1000.nc', '1001-2000.nc', '2001-3000.nc', '3001-4000.nc']

basin_datasets = {}
for s, SSP in enumerate(SSPs):
    which_ssp = SSPs[s]
    basin_datasets[which_ssp] = {}  # Initialize basin_datasets with the ssp key
    
    fpath1 = '/11/mass-annual/R11_mass_annual_c2_ba1_1set_2000_2100-{}-Batch-'.format(which_ssp)

    for basin, glacier_list in basin_gls.items():
        ## loop over them all, drop the irrelevant IDs, and concatenate the result
        ds_list = []
        for b in batches:
            ds_temp = xr.open_dataset(fpath0 + fpath1 + b)
            try:
                ds_filtered = ds_temp.where(ds_temp.RGIId.isin(glacier_list), drop=True)
                ds_list.append(ds_filtered)
            except ValueError: ## happens if there are no glaciers from this batch in the selected region
                continue
        basin_datasets[which_ssp][basin] = xr.concat(ds_list, dim='glacier')

In [ ]:
#Now we find basin-wide volume by summing within basin and converting kg to km^3
basins = ['RHINE', 'RHONE', 'PO','DANUBE']
basin_volume_ds = {}
for s, SSP in enumerate(SSPs):
    which_ssp = SSPs[s]
    basin_volume_ds[which_ssp] = {}
    for basin in basins:
        basin_volume_ds[SSP][basin] = basin_datasets[SSP][basin].glac_mass_annual.sum(dim='glacier')*1e-12

In [ ]:
#We find the annual melt via the annual volume diff
basins = ['RHINE', 'RHONE', 'PO','DANUBE']
basin_melt_ds = {}
for s, SSP in enumerate(SSPs):
    which_ssp = SSPs[s]
    basin_melt_ds[which_ssp] = {}
    for basin in basins:
        basin_melt_ds[SSP][basin] = basin_volume_ds[SSP][basin].diff(dim = 'year')*-1

In [ ]:
#Plotting Volume change for all GCMs by basin
from cycler import cycler

scenarios = ['ssp126', 'ssp245', 'ssp370', 'ssp585']
basins = ['RHINE', 'RHONE', 'PO','DANUBE']

#Creating color scheme
color_map = plt.colormaps['bone']
line_colors = color_map(np.linspace(0.1, 0.9, num = 12))
custom_cycler = cycler(color = line_colors)

#Plotting all data
fig, axs = plt.subplots(len(basins), len(scenarios), figsize=(12, 10), sharex=True)
for s, SSP in enumerate(scenarios):
    which_ssp = SSPs[s]
    for b, basin in enumerate(basins):
        basin_volume_ds[which_ssp][basin].plot(hue='model', ax=axs[b, s], color=axs[b, s].set_prop_cycle(custom_cycler), add_legend=False)
        if b == 3:
            for sub_s in range(4):  # Use a different variable name for the inner loop
                axs[sub_b, s].set_xlabel('Year')
        else:
            axs[b, s].set_xlabel(None)
        if s == 0:
            for b, basin in enumerate(basins):  # Use a different variable name for the inner loop
                axs[b, s].set_ylabel(str(basins[b])+' Volume $[km^3]$')
        else:
            axs[b, s].set_ylabel(None)

#Adding legend, titles, etc       
axs[0,0].legend(['BCC-CSM2-MR','CESM2','CESM2-WACCM','EC-Earth3','EC-Earth3-Veg','FGOALS-f3-L','GFDL-ESM4',
              'INM-CM4-8','INM-CM5-0','MPI-ESM1-2-HR','MRI-ESM2-0', 'NorESM2-MM'],bbox_to_anchor=(4.96, 1.64), ncol=6)
plt.subplots_adjust(top=0.85, wspace=0.3, hspace=0.2)
plt.suptitle('Total Glacial Volume within Major River Basins in Central Europe–PyGEM')
plt.title('SSP 126                                   SSP 245                                  SSP 370                              SSP 585', x=-1.41, y=4.72)


In [ ]:
#Plotting Annual Melt for all GCMs by basin
#Choose a 30 yr rolling mean for readibility
from cycler import cycler

scenarios = ['ssp126', 'ssp245', 'ssp370', 'ssp585']
basins = ['RHINE', 'RHONE', 'PO','DANUBE']

#Creating color scheme
color_map = plt.colormaps['bone']
line_colors = color_map(np.linspace(0.1, 0.9, num = 12))
custom_cycler = cycler(color = line_colors)

#Plotting all data
fig, axs = plt.subplots(len(basins), len(scenarios), figsize=(12, 10), sharex=True)
for s, SSP in enumerate(scenarios):
    which_ssp = SSPs[s]
    for b, basin in enumerate(basins):
        basin_melt_ds[which_ssp][basin].rolling(year=30).mean().plot(hue='model', ax=axs[b, s], color=axs[b, s].set_prop_cycle(custom_cycler), add_legend=False)
        if b == 3:
            for sub_s in range(4):  # Use a different variable name for the inner loop
                axs[sub_b, s].set_xlabel('Year')
        else:
            axs[b, s].set_xlabel(None)
        if s == 0:
            for b, basin in enumerate(basins):  # Use a different variable name for the inner loop
                axs[b, s].set_ylabel(str(basins[b])+' Volume $[km^3]$')
        else:
            axs[b, s].set_ylabel(None)

#Adding legend, titles, etc       
axs[0,0].legend(['BCC-CSM2-MR','CESM2','CESM2-WACCM','EC-Earth3','EC-Earth3-Veg','FGOALS-f3-L','GFDL-ESM4',
              'INM-CM4-8','INM-CM5-0','MPI-ESM1-2-HR','MRI-ESM2-0', 'NorESM2-MM'],bbox_to_anchor=(4.96, 1.64), ncol=6)
plt.subplots_adjust(top=0.85, wspace=0.3, hspace=0.2)
plt.suptitle('Annual Glacial Melt in Major Central European River Basins–PyGEM')
plt.title('SSP 126                                   SSP 245                                SSP 370                              SSP 585', x=-1.41, y=4.72)
